# Coffe Shop Purchase Analytics

Artificial model consists on analysis of approximatelly 1,000 transactions in coffe shop.

All data is faked. So the main purpose is to detect some strong correlations, visualize them and construct statistics-driven predictions

Breaking in pieces, we're interested in such visualization params:

1) Average customer order amount
2) Order-by-time dispertion
3) Common meals combinations

Prediction model is dedicatated to:

1) Calculate median profit distributed by time range
2) Offer best-seller meal

With aim of high accuracy, with obvious, fast and robust implementation Tensorflow+Keras framework are used in research


In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense